In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
bg_df = pd.read_csv("../Datasets/boardgame.csv")
bg_df

,boardgame_id,user_id,average,bayesaverage,tags
0,30549,109006,7.58896,7.48669,in pandemic several virulent diseases have bro...
1,822,108776,7.41837,7.30857,carcassonne is a tileplacement game in which t...
2,13,108064,7.13598,6.96965,in catan formerly the settlers of catan player...
3,68448,90021,7.73515,7.63355,you are the leader of one of the 7 great citie...
4,36218,81582,7.61000,7.49912,quotyou are a monarch like your parents before...
...,...,...,...,...,...
21625,296892,30,7.36667,5.53114,it has come time to honor the traditions of th...
21626,217378,30,6.61333,5.51748,babel is a balancing game in which players mak...
21627,18063,30,5.00000,5.48821,from the rulesplayers are prospectors searchin...
21628,10052,30,5.80000,5.50484,each player takes a position adjacent to one o...


In [3]:
ratings_df = pd.read_csv("../Datasets/ratings.csv")
ratings_df

,user_name,rating,comment,boardgame_id,boardgame_name
0,Staxston,9.2,Only game I really play solo.,171835,Posthuman
1,Sheylon,3.0,Cute pieces but this isn't much of a game. Rol...,5824,The Kids of Catan
2,jens_hoppe,3.0,Initial impression. Takes heavy-weight German-...,25554,Notre Dame
3,bcnevan,5.0,Shores of Tripoli succeeds in some ways. It is...,237860,The Shores of Tripoli
4,ruelgaviola,8.0,Donated,17329,Animal Upon Animal
...,...,...,...,...,...
673718,Andy Parsons,5.0,I found the theme of Say Bye... quite appealin...,140795,Say Bye to the Villains
673719,PHIL4477,4.0,Another trivia game. I bought it because the b...,32441,Bezzerwizzer
673720,EmeraldYam,7.0,(As of [b]3[/b] plays) Quite original card-bas...,125315,FlowerFall
673721,PaleHorseRider,6.0,Good version of Apples to Apples. It has cons...,104234,Sour Apples to Apples


In [5]:
ratings_df["boardgame_name"].unique().tolist()

['Posthuman',
 'The Kids of Catan',
 'Notre Dame',
 'The Shores of Tripoli',
 'Animal Upon Animal',
 'Patrician',
 'Fauna',
 'Archaeology: The Card Game',
 'Marvel Dice Masters: Uncanny X-Men',
 'Secrets of the Lost Tomb',
 'Power Grid: Factory Manager',
 'Sword & Sorcery',
 'New Salem',
 'Igloo Pop',
 'Arkham Horror: The Card Game',
 'Forbidden Desert',
 'Ginkgopolis',
 'Saint Petersburg',
 'Endeavor',
 'Star Wars: X-Wing Miniatures Game',
 'Lost Cities',
 'Cloudspire',
 'Alchemist',
 'Blood Bowl (Third Edition)',
 'Ascension: Darkness Unleashed',
 'Charterstone',
 'Android: Netrunner',
 'Aristocracy',
 'Castle Panic',
 'RAF: The Battle of Britain 1940',
 'Aether Captains',
 'Pajarracos',
 'Eclipse',
 'Days of Decision',
 'Chess',
 'Hotel Tycoon',
 'Zombie Dice',
 'Timeline: Discoveries',
 'Speed Circuit',
 'Herbaceous',
 'Lisboa',
 'Last Night on Earth: The Zombie Game',
 'Heroica: Fortaan',
 'Vs System',
 'Drop It',
 'Terraforming Mars',
 'The Ark of the Covenant',
 'Discoveries: Th

In [4]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 673723 entries, 0 to 673722
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_name       673723 non-null  object 
 1   rating          673723 non-null  float64
 2   comment         673723 non-null  object 
 3   boardgame_id    673723 non-null  int64  
 4   boardgame_name  673723 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 25.7+ MB


In [5]:
# Số len của boardgame_id có khoảng 20776 vậy sample dataset cũng có thể xem là bao quát cho toàn bộ dữ liệu ban đầu
len(ratings_df["boardgame_id"].unique())


20776

### Handle the consequent text in the comment column

In [6]:
def remove_space(word):
    l = []
    for i in word:
        l.append(i.replace(" ", "").replace("-", "").replace(".", ""))
    return l

## Delete some especial letter and don't LAtinh Word
import re
def filter_non_latin(list_word):
    l = []
    for text in list_word:
        if text != "Missing":
                # loại bỏ những kí tự có &#
                text = re.sub(r'&#[0-9]+', '', text)
                # Loại bỏ các kí tự không phải Latin
                latin_only = re.sub(r'[^\x00-\x7F]+', ' ', text)
                # Loại bỏ các kí tự đặc biệt như '#'
                clean_text = re.sub(r'[^A-Za-z0-9\s]', '', latin_only)
                l.append(clean_text)
    return l

def filter_number_appear(list_word):
    l = []
    for text in list_word:
                # loại bỏ những kí tự có số > chuỗi 4 vd: từ 1000
                text = re.sub(r'[0-9][0-9][0-9][0-9]+', '', text)
                l.append(text)
    return l

In [7]:
### Xử lí chuỗi 
ratings_df["comment"] = ratings_df["comment"].apply(lambda x:x.split())

In [8]:
ratings_df["comment"] = ratings_df["comment"].apply(remove_space)
ratings_df["comment"] = ratings_df["comment"].apply(filter_non_latin)
ratings_df["comment"] = ratings_df["comment"].apply(filter_number_appear)
ratings_df["comment"] = ratings_df["comment"].apply(lambda x: " ".join(x))
ratings_df["comment"] = ratings_df["comment"].apply(lambda x: x.lower())
ratings_df

,user_name,rating,comment,boardgame_id,boardgame_name
0,Staxston,9.2,only game i really play solo,171835,Posthuman
1,Sheylon,3.0,cute pieces but this isnt much of a game roll ...,5824,The Kids of Catan
2,jens_hoppe,3.0,initial impression takes heavyweight germansty...,25554,Notre Dame
3,bcnevan,5.0,shores of tripoli succeeds in some ways it is ...,237860,The Shores of Tripoli
4,ruelgaviola,8.0,donated,17329,Animal Upon Animal
...,...,...,...,...,...
673718,Andy Parsons,5.0,i found the theme of say bye quite appealing t...,140795,Say Bye to the Villains
673719,PHIL4477,4.0,another trivia game i bought it because the bi...,32441,Bezzerwizzer
673720,EmeraldYam,7.0,as of b3b plays quite original cardbased dexte...,125315,FlowerFall
673721,PaleHorseRider,6.0,good version of apples to apples it has conseq...,104234,Sour Apples to Apples


In [9]:
### Do sẽ có nhiều boardgame_id được nhiều user_name comment vì vậy ta phải tổng hợp các comment của boardgame_id đó lại 

In [10]:
ratings_df = ratings_df.sort_values(by= "boardgame_id")
ratings_df

,user_name,rating,comment,boardgame_id,boardgame_name
43714,Kalvin,8.0,one of my favorite games i need to think of a ...,1,Die Macher
289051,crashcrosby,9.0,every time we play this we slip into german ac...,1,Die Macher
167704,MichaelB,9.0,finally played a really good game with few fla...,1,Die Macher
530167,slowcorner,8.0,long but surprisingly good political game ther...,1,Die Macher
480864,djflippy,10.0,this title is quite simply one of the best exa...,1,Die Macher
...,...,...,...,...,...
57134,antiedipo,9.0,i love design this game d i love to play it wi...,350992,Neuroriders
234859,nita_casten,9.0,entretenido p,350992,Neuroriders
8322,brosDe,8.0,de momento solo he jugado una nica partida al...,350992,Neuroriders
490752,Octo_,8.0,me cost un poco tomar el ritmo del juego pero...,350992,Neuroriders


In [11]:
ratings_df["comment"]

43714     one of my favorite games i need to think of a ...
289051    every time we play this we slip into german ac...
167704    finally played a really good game with few fla...
530167    long but surprisingly good political game ther...
480864    this title is quite simply one of the best exa...
                                ...                        
57134     i love design this game d i love to play it wi...
234859                                        entretenido p
8322      de momento solo he jugado una  nica partida al...
490752    me cost  un poco tomar el ritmo del juego pero...
552416    played in tts with the designer of the game i ...
Name: comment, Length: 673723, dtype: object

In [12]:
concatenated_text = defaultdict(str)
for bg_unique in ratings_df["boardgame_id"].unique():
    test_cm = ratings_df[ratings_df["boardgame_id"] == bg_unique]["comment"]
    test_l = len(test_cm)
    str_test = ""
    for i in range(test_l):
        str_test += list(test_cm)[i] + " "
    concatenated_text[bg_unique] = str_test
    print(bg_unique)
# print(str_test)
new_data = {'boardgame_id': list(concatenated_text.keys()), 'comment_tags': list(concatenated_text.values())}
new_df = pd.DataFrame(new_data)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
34
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
57
58
59
60
61
62
63
64
66
67
68
69
70
71
72
73
74
75
76
77
78
79
82
83
84
85
87
88
89
90
91
93
94
95
96
97
98
99
101
102
103
104
105
106
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
124
125
126
127
128
130
132
133
134
135
136
137
139
140
141
142
144
145
146
147
148
150
151
152
153
154
155
156
157
162
163
164
165
166
167
168
169
170
171
172
174
175
176
178
179
180
181
182
183
185
186
188
189
190
192
193
194
195
196
197
198
199
200
201
202
204
206
207
208
210
211
214
215
216
217
218
219
220
221
222
223
224
225
226
228
229
230
231
232
234
235
237
239
240
241
242
243
244
246
247
248
249
250
252
254
255
256
257
258
259
260
261
263
264
265
267
268
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
307
308
309
310
311
312
313
314
315
316


In [13]:
new_df

,boardgame_id,comment_tags
0,1,one of my favorite games i need to think of a ...
1,2,classic with cool components 1 play 3 players ...
2,3,if you only bye ten games in youre life i thin...
3,4,an excellent game with a high degree of player...
4,5,very tense game liked it a lot we play with hi...
...,...,...
20771,348955,the designer jeff kornberg hit this one out of...
20772,349131,yes its snappy but tends to be frustrating the...
20773,349805,worthingtons introduction resurrection o...
20774,350736,such a fun game and great concept of selling i...


In [14]:
new_df = new_df.rename(columns={"id": "boardgame_id" , "text_concatenated" : "comment_tags"})
new_df

,boardgame_id,comment_tags
0,1,one of my favorite games i need to think of a ...
1,2,classic with cool components 1 play 3 players ...
2,3,if you only bye ten games in youre life i thin...
3,4,an excellent game with a high degree of player...
4,5,very tense game liked it a lot we play with hi...
...,...,...
20771,348955,the designer jeff kornberg hit this one out of...
20772,349131,yes its snappy but tends to be frustrating the...
20773,349805,worthingtons introduction resurrection o...
20774,350736,such a fun game and great concept of selling i...


In [15]:
# Lấy tên boardgame_id
new_df_1 = new_df.merge(ratings_df, on = "boardgame_id", how="inner")
new_df_1

,boardgame_id,comment_tags,user_name,rating,comment,boardgame_name
0,1,one of my favorite games i need to think of a ...,Kalvin,8.0,one of my favorite games i need to think of a ...,Die Macher
1,1,one of my favorite games i need to think of a ...,crashcrosby,9.0,every time we play this we slip into german ac...,Die Macher
2,1,one of my favorite games i need to think of a ...,MichaelB,9.0,finally played a really good game with few fla...,Die Macher
3,1,one of my favorite games i need to think of a ...,slowcorner,8.0,long but surprisingly good political game ther...,Die Macher
4,1,one of my favorite games i need to think of a ...,djflippy,10.0,this title is quite simply one of the best exa...,Die Macher
...,...,...,...,...,...,...
673718,350992,played a prototype of the game so the rating w...,antiedipo,9.0,i love design this game d i love to play it wi...,Neuroriders
673719,350992,played a prototype of the game so the rating w...,nita_casten,9.0,entretenido p,Neuroriders
673720,350992,played a prototype of the game so the rating w...,brosDe,8.0,de momento solo he jugado una nica partida al...,Neuroriders
673721,350992,played a prototype of the game so the rating w...,Octo_,8.0,me cost un poco tomar el ritmo del juego pero...,Neuroriders


In [16]:
new_df_1 = new_df_1[["boardgame_id", "comment_tags", "boardgame_name"]]
new_df_1

,boardgame_id,comment_tags,boardgame_name
0,1,one of my favorite games i need to think of a ...,Die Macher
1,1,one of my favorite games i need to think of a ...,Die Macher
2,1,one of my favorite games i need to think of a ...,Die Macher
3,1,one of my favorite games i need to think of a ...,Die Macher
4,1,one of my favorite games i need to think of a ...,Die Macher
...,...,...,...
673718,350992,played a prototype of the game so the rating w...,Neuroriders
673719,350992,played a prototype of the game so the rating w...,Neuroriders
673720,350992,played a prototype of the game so the rating w...,Neuroriders
673721,350992,played a prototype of the game so the rating w...,Neuroriders


In [17]:
new_df_1 = new_df_1.drop_duplicates()

In [18]:
new_df_1

,boardgame_id,comment_tags,boardgame_name
0,1,one of my favorite games i need to think of a ...,Die Macher
293,2,classic with cool components 1 play 3 players ...,Dragonmaster
337,3,if you only bye ten games in youre life i thin...,Samurai
958,4,an excellent game with a high degree of player...,Tal der Könige
981,5,very tense game liked it a lot we play with hi...,Acquire
...,...,...,...
673695,348955,the designer jeff kornberg hit this one out of...,Rock Paper Scissors: Deluxe Edition
673698,349131,yes its snappy but tends to be frustrating the...,Splitter
673700,349805,worthingtons introduction resurrection o...,Bismarck Solitaire
673703,350736,such a fun game and great concept of selling i...,Voyages


In [19]:
cb_df = bg_df.merge(new_df_1, on="boardgame_id", how="inner")
cb_df

,boardgame_id,user_id,average,bayesaverage,tags,comment_tags,boardgame_name
0,30549,109006,7.58896,7.48669,in pandemic several virulent diseases have bro...,super coop great cooperative game lots of fun ...,Pandemic
1,822,108776,7.41837,7.30857,carcassonne is a tileplacement game in which t...,played a lighthearted game that anyone can pla...,Carcassonne
2,13,108064,7.13598,6.96965,in catan formerly the settlers of catan player...,this is the only game that ive played hundreds...,Catan
3,68448,90021,7.73515,7.63355,you are the leader of one of the 7 great citie...,i still love this game most with no expansions...,7 Wonders
4,36218,81582,7.61000,7.49912,quotyou are a monarch like your parents before...,26 d after playing thunderstone dominion feels...,Dominion
...,...,...,...,...,...,...,...
20583,16874,30,7.32533,5.52472,quotvince lombardis gamequot is a football sim...,this is long out of print although it can be f...,Vince Lombardi's Game
20584,3589,30,4.86667,5.48788,from the boxthe beast wars have begun choose s...,fun game played it a bit in college actually s...,Beast Wars Transformers Mutating Card Game
20585,296892,30,7.36667,5.53114,it has come time to honor the traditions of th...,the only traitorsocial deduction i have actual...,Sacred Rites
20586,10052,30,5.80000,5.50484,each player takes a position adjacent to one o...,actually its my 4year olds he loves it,Bongo Kongo


In [20]:
cb_df["main_tags"] = cb_df["tags"] + cb_df["comment_tags"]
cb_df

,boardgame_id,user_id,average,bayesaverage,tags,comment_tags,boardgame_name,main_tags
0,30549,109006,7.58896,7.48669,in pandemic several virulent diseases have bro...,super coop great cooperative game lots of fun ...,Pandemic,in pandemic several virulent diseases have bro...
1,822,108776,7.41837,7.30857,carcassonne is a tileplacement game in which t...,played a lighthearted game that anyone can pla...,Carcassonne,carcassonne is a tileplacement game in which t...
2,13,108064,7.13598,6.96965,in catan formerly the settlers of catan player...,this is the only game that ive played hundreds...,Catan,in catan formerly the settlers of catan player...
3,68448,90021,7.73515,7.63355,you are the leader of one of the 7 great citie...,i still love this game most with no expansions...,7 Wonders,you are the leader of one of the 7 great citie...
4,36218,81582,7.61000,7.49912,quotyou are a monarch like your parents before...,26 d after playing thunderstone dominion feels...,Dominion,quotyou are a monarch like your parents before...
...,...,...,...,...,...,...,...,...
20583,16874,30,7.32533,5.52472,quotvince lombardis gamequot is a football sim...,this is long out of print although it can be f...,Vince Lombardi's Game,quotvince lombardis gamequot is a football sim...
20584,3589,30,4.86667,5.48788,from the boxthe beast wars have begun choose s...,fun game played it a bit in college actually s...,Beast Wars Transformers Mutating Card Game,from the boxthe beast wars have begun choose s...
20585,296892,30,7.36667,5.53114,it has come time to honor the traditions of th...,the only traitorsocial deduction i have actual...,Sacred Rites,it has come time to honor the traditions of th...
20586,10052,30,5.80000,5.50484,each player takes a position adjacent to one o...,actually its my 4year olds he loves it,Bongo Kongo,each player takes a position adjacent to one o...


In [21]:
cb_df["main_tags"][0]

'in pandemic several virulent diseases have broken out simultaneously all over the world the players are diseasefighting specialists whose mission is to treat disease hotspots while researching cures for each of four plagues before they get out of handthe game board depicts several major population centers on earth on each turn a player can use up to four actions to travel between cities treat infected populaces discover a cure or build a research station a deck of cards provides the players with these abilities but sprinkled throughout this deck are epidemic cards that accelerate and intensify the diseases activity a second separate deck of cards controls the quotnormalquot spread of the infectionstaking a unique role within the team players must plan their strategy to mesh with their specialists strengths in order to conquer the diseases for example the operations expert can build research stations which are needed to find cures for the diseases and which allow for greater mobility b

In [22]:
cb_df = cb_df[["boardgame_id", "boardgame_name", "main_tags"]]
cb_df

,boardgame_id,boardgame_name,main_tags
0,30549,Pandemic,in pandemic several virulent diseases have bro...
1,822,Carcassonne,carcassonne is a tileplacement game in which t...
2,13,Catan,in catan formerly the settlers of catan player...
3,68448,7 Wonders,you are the leader of one of the 7 great citie...
4,36218,Dominion,quotyou are a monarch like your parents before...
...,...,...,...
20583,16874,Vince Lombardi's Game,quotvince lombardis gamequot is a football sim...
20584,3589,Beast Wars Transformers Mutating Card Game,from the boxthe beast wars have begun choose s...
20585,296892,Sacred Rites,it has come time to honor the traditions of th...
20586,10052,Bongo Kongo,each player takes a position adjacent to one o...


In [23]:
cb_df.to_csv("../Datasets/content_based_dataset.csv", index=False)